In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC

from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier


In [36]:
# what to do:
# knn svm logistic random_forest 
# us / not us 


In [37]:
from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

In [38]:
# consolidating income col values
y[y['income'] == '<=50K.'] = '<=50K'
y[y['income'] == '>50K.'] = '>50K'
y['income'] = y['income'].apply(lambda x: 1 if x == '<=50K' else 0)

/var/folders/t0/08jdm81x5hl2xjtdyy2hjzmh0000gn/T/ipykernel_7331/3281958407.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y['income'] == '<=50K.'] = '<=50K'
/var/folders/t0/08jdm81x5hl2xjtdyy2hjzmh0000gn/T/ipykernel_7331/3281958407.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y['income'] == '<=50K.'] = '<=50K'
/var/folders/t0/08jdm81x5hl2xjtdyy2hjzmh0000gn/T/ipykernel_7331/3281958407.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

#### Missing Values

In [39]:
# finds all rows that has a "?" or missing values, then filters dataframe to get rows that do not have those values
cols = X.columns
has = ~(X['age'] == X['age'])
for col in cols:
    has = ((has) | (X[col] =='?') | (X[col].isna()))

X = X[~has]
y = y[~has]

In [40]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


# Logistic Regression Model

In [41]:
preproc = ColumnTransformer(
    transformers = [
        ("categorical", OneHotEncoder(handle_unknown='ignore', sparse_output = False), ["race", "sex", "occupation"]),
        ("numerical", "passthrough", ["age", "hours-per-week", "education-num"])
    ],
    remainder = "drop"
)

In [42]:
plr = Pipeline([
    ("preproc", preproc),
    ("model", LogisticRegression())
])

plr.fit(X_train, y_train)

/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['race', 'sex',
                                                   'occupation']),
                                                 ('numerical', 'passthrough',
                                                  ['age', 'hours-per-week',
                                                   'education-num'])])),
                ('model', LogisticRegression())])

In [43]:
predicted = plr.predict(X_test)

In [44]:
# accuracy
(pd.Series(predicted) == y_test.reset_index()['income']).mean()

0.8078425591508808

In [45]:
# recall
recall_score(y_test, predicted)

0.9368193858530364

In [46]:
# f1 score
f1_score(y_test, predicted)

0.8798894263994471

In [49]:
# AUC-ROC
roc_auc_score(y_test, predicted)

0.6775086852205312

# Support Vector Machine

In [50]:
preproc = ColumnTransformer(
    transformers = [
        ("categorical", OneHotEncoder(handle_unknown='ignore', sparse_output = False), ["race", "sex", "occupation"]),
        ("numerical", "passthrough", ["age", "hours-per-week", "education-num"])
    ],
    remainder = "drop"
)

In [51]:
plr = Pipeline([
    ("preproc", preproc),
    ("model", SVC())
])

plr.fit(X_train, y_train)

/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/utils/validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['race', 'sex',
                                                   'occupation']),
                                                 ('numerical', 'passthrough',
                                                  ['age', 'hours-per-week',
                                                   'education-num'])])),
                ('model', SVC())])

In [52]:
predicted = plr.predict(X_test)

In [53]:
# accuracy
(pd.Series(predicted) == y_test.reset_index()['income']).mean()

0.8019458981351809

In [54]:
# recall
recall_score(y_test, predicted)

0.9600706367114686

In [55]:
# f1 score
f1_score(y_test, predicted)

0.8792847836830047

In [56]:
# AUC-ROC
roc_auc_score(y_test, predicted)

0.6421574286106246

# Random Forrest

In [57]:
preproc = ColumnTransformer(
    transformers = [
        ("categorical", OneHotEncoder(handle_unknown='ignore', sparse_output = False), ["race", "sex", "occupation"]),
        ("numerical", "passthrough", ["age", "hours-per-week", "education-num"])
    ],
    remainder = "drop"
)

In [83]:
plr = Pipeline([
    ("preproc", preproc),
    ("model", RandomForestClassifier(max_depth=10, random_state=0))
])

plr.fit(X_train, y_train)

/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['race', 'sex',
                                                   'occupation']),
                                                 ('numerical', 'passthrough',
                                                  ['age', 'hours-per-week',
                                                   'education-num'])])),
                ('model',
                 RandomForestClassifier(max_depth=10, random_state=0))])

In [84]:
predicted = plr.predict(X_test)

In [85]:
# accuracy
(pd.Series(predicted) == y_test.reset_index()['income']).mean()

0.8127810127515295

In [86]:
# recall
recall_score(y_test, predicted)

0.9508486216030609

In [87]:
# f1 score
f1_score(y_test, predicted)

0.8841452289728152

In [88]:
# AUC-ROC
roc_auc_score(y_test, predicted)

0.6732607067707066

# LightGBM

In [180]:
from lightgbm import LGBMClassifier

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [181]:
categorical = ["race", "sex", "occupation", "workclass", "education", "marital-status", "relationship", "native-country"]
preproc = ColumnTransformer(
    transformers = [
        ("categorical", OneHotEncoder(handle_unknown='ignore', sparse_output = True), categorical),
        ("numerical", "passthrough", ["age", "hours-per-week", "education-num", "fnlwgt", "capital-gain", "capital-loss" ])
    ],
    remainder = "drop"
)

In [247]:
plr = Pipeline([
    ("preproc", preproc),
    ("model", LGBMClassifier(max_depth = -1, learning_rate = .01, n_estimators = 1000, n_jobs = -1))
])
plr.fit(X_train, y_train)

/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Info] Number of positive: 23845, number of negative: 7810
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004024 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 771
[LightGBM] [Info] Number of data points in the train set: 31655, number of used features: 93
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.753278 -> initscore=1.116170
[LightGBM] [Info] Start training from score 1.116170


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('categorical',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['race', 'sex', 'occupation',
                                                   'workclass', 'education',
                                                   'marital-status',
                                                   'relationship',
                                                   'native-country']),
                                                 ('numerical', 'passthrough',
                                                  ['age', 'hours-per-week',
                                                   'education-num', 'fnlwgt',
                                                   'capital-gain',
                                                   'capital-loss'])])),
                ('model',
                 LGBMClassifier(learning_rate=0.01, n_estimators=1000,
                                n_jobs=-1))])

In [248]:
predicted = plr.predict(X_test)

/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [249]:
# accuracy
(pd.Series(predicted) == y_test.reset_index()['income']).mean()

0.8664406279943981

In [250]:
# recall
recall_score(y_test, predicted)

0.9381453436916117

In [251]:
# f1 score
f1_score(y_test, predicted)

0.913268236645606

In [252]:
# AUC-ROC
roc_auc_score(y_test, predicted)

0.7949996877375068

In [ ]:
#gridsearch cv
from sklearn.model_selection import GridSearchCV
gridParams = {
    'learning_rate': [0.005, 0.01],
    'n_estimators': [8,16,24],
    'num_leaves': [6,8,12,16], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['gbdt', 'dart'], # for better accuracy -> try dart
    'objective' : ['binary'],
    'max_bin':[255, 510], # large max_bin helps improve accuracy but might slow down training progress
    'random_state' : [500],
    'colsample_bytree' : [0.64, 0.65, 0.66],
    'subsample' : [0.7,0.75],
    'reg_alpha' : [1,1.2],
    'reg_lambda' : [1,1.2,1.4],
    }
grid_search = GridSearchCV(estimator=LGBMClassifier(), param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1, verbose=1)

grid_search.fit(preproc.fit_transform(X_train), y_train)

Fitting 3 folds for each of 162 candidates, totalling 486 fits


/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vect

[LightGBM] [Info] Number of positive: 15897, number of negative: 5206
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.022017 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 740
[LightGBM] [Info] Number of data points in the train set: 21103, number of used features: 88
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.753305 -> initscore=1.116319
[LightGBM] [Info] Start training from score 1.116319
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn

[LightGBM] [Info] Number of positive: 15896, number of negative: 5207
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.020989 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 740
[LightGBM] [Info] Number of data points in the train set: 21103, number of used features: 87
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.753258 -> initscore=1.116064
[LightGBM] [Info] Start training from score 1.116064
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -

/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:93: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/Users/calwinli/miniforge3/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn